# Agnostic methods for liquid handling


## Setting up 

Check if the correct environment is in use:

In [ ]:
!echo $VIRTUAL_ENV

import os, sys
print("Current working directory: " + os.getcwd())
print("Using python: " + sys.executable)

To install PLR from a local repository, run the following from the repository's directory:

In [ ]:
# Note that adding "fw" will install some required dependencies. A plain "." will not.
%pip install .[fw]

?

In [ ]:
%load_ext autoreload
%autoreload 2

Basic imports:

In [ ]:
# Basic imports:
# - liquid handler
from pylabrobot.liquid_handling import LiquidHandler
# - backend
from pylabrobot.liquid_handling.backends import STAR
# - deck
from pylabrobot.resources.hamilton import STARLetDeck

## Writing a custom deck

In [ ]:
import textwrap
from typing import Optional, Callable

from pylabrobot.resources import Coordinate, Deck, Trash


class SilverDeck(Deck):
  """ (Ag)nostic deck object.

  Boilerplate code written by Rick: https://forums.pylabrobot.org/t/writing-a-new-backend-agnosticity/844/16
  """

  def __init__(self,
               name: str= "silver_deck",
               # TODO: Update default size.
               size_x: float = 250,
               size_y: float = 350,
               size_z: float = 200,
               resource_assigned_callback: Optional[Callable] = None,
               resource_unassigned_callback: Optional[Callable] = None,
               # TODO: Update default origin.
               origin: Coordinate = Coordinate(0, 0, 0),
               # TODO: Update default trash location.
               trash_location: Coordinate = Coordinate(x=82.84, y=53.56, z=5),
               no_trash: bool = False):

    # Run init from the base Deck class.
    super().__init__(
      name=name,
      size_x=size_x, size_y=size_y, size_z=size_z,
      resource_assigned_callback=resource_assigned_callback,
      resource_unassigned_callback=resource_unassigned_callback,
      origin=origin)

    # TODO: write your init code, for example assign a "trash" resource:
    if not no_trash:
      self._assign_trash(location=trash_location)

  def _assign_trash(self, location: Coordinate):
    """ Assign the trash area to the deck. """

    trash = Trash(
      name="trash",
      # TODO: Update default dimensions.
      size_x=80,
      size_y=120,
      size_z=50
    )

    self.assign_child_resource(trash, location=location)

  def summary(self) -> str:
    """ Get a summary of the deck.

    >>> print(deck.summary())

    TODO: <write some printable ascii representation of the deck's current layout>
    """

    return textwrap.dedent(f"""\
      +---------------------+
      |                     |
      |        TODO         |
      |                     |
      +---------------------+
    """)


Instantiate a new deck object:

In [ ]:
deck = SilverDeck(name="basic deck 1")

Stuff can be added to the deck, which in PLR jargon is the same as "assigning a child resource" to the deck.

This is typically accomplished by calling `assign_child_resource`, a method from the Resource class,
which is also the baseclass for Deck.

This means that our deck has a list of resources, which are considered children of the deck resource:

In [ ]:
deck.children

Lets add another resource, a well plate:

In [ ]:
# Import the resource class
from pylabrobot.resources import Cos_96_DW_1mL, LTF_L

# Create an instance
well_plate = Cos_96_DW_1mL(name='plate_01')
tip_rack = LTF_L(name="tip rack 1")

Note that the well plate is also a resource, and also has children. Lets print the first five children, and realize that it has been populated with "Well" type resources:

In [ ]:
well_plate.children[:5]

Likewise, the tip rack has been populated with "tip spot" resources, where tips can be placed.

In [ ]:
tip_rack.children[:3]

The tip spots on the rack can be filled with tips:

In [ ]:
# tip_rack.fill()

Now we will assign the well plate resource as a child of our deck resource using `assign_child_resource`.

The `assign_child_resource` function inherited from base `Resource` class accepts an optional `location` parameter, with type `Coordinate` (another type of PLR object). If passed, it is used to override the `location` attribute of the Resource we want to add.

 Lets have a look:

In [ ]:
# Note that the well has no default location.
print(well_plate.location)

In [ ]:
# Since some robots have "slots" or "rails", the syntax will vary.
# For example, the pyhamilton deck has overriden the "assign_child_resource"
# method from "Resource", such that it accepts a "rail" parameter instead
# of a location/Coordinate.
deck.assign_child_resource(well_plate, location=Coordinate(10, 10, 0))

# Do the same for the tip rack.
deck.assign_child_resource(tip_rack, location=Coordinate(10, 100, 0))

In [ ]:
# Note that the well now has location.
print(well_plate.location)

In [ ]:
# Note that the well plate now has a parent
print(well_plate.parent)

In [ ]:
# Note that the deck now has an additional child.
deck.children

## Deck Serialization

Lets see what serialization outputs:

In [ ]:
import json
data = deck.serialize()
with open('data/deck.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, indent=4)


## A couple ways of breaking PLR

Lets try to break PLR by assigning the same object again:

In [ ]:
try:
  deck.assign_child_resource(well_plate, location=Coordinate(10, 10, 0), reassign=False)
except Exception as e:
  print(f"Error: {e}")
else:
  print(f"No error :/")

Lets try to break PLR by re-assigning the same child to another parent, with the reassing disabled:

In [ ]:
try:
  deck2 = SilverDeck(name="second deck")
  deck2.assign_child_resource(well_plate, location=Coordinate(10, 10, 0), reassign=False)
except Exception as e:
  print(f"Error: {e}")
else:
  print(f"No error :/")

Note that `reassing=True` (the default) actually reassigns the resource to a different parent.

In [ ]:
try:
  deck2 = SilverDeck(name="second deck")
  deck2.assign_child_resource(well_plate, location=Coordinate(10, 10, 0))
except Exception as e:
  print(f"Error: {e}")


In [ ]:
well_plate.parent.name

Lets try to break PLR by assigning a Resource to itself:

In [ ]:
try:
  well_plate.assign_child_resource(well_plate, location=Coordinate(10, 10, 0))
except Exception as e:
  print(f"Error: {e}")
else:
  print(f"No error :/")

# Pipettin Writer Objects

JSON definition of a workspace: how does it match a PLR "deck"?

## Object mapping

I'm not sure if there is a 1-to-1 mapping between Pipttin's and PLR's objects.

I'll start from here:

| Pipettin Writer | PyLabRobot |  PW -> PLR  |  PW <- PLR  |
|-----------------|------------|-------------|-------------|
| Workspace       | Deck       |  Minimal    |  -          |
| BUCKET          | Trash      |  -          |  -          |
| TUBE_RACK       | Plate      |  -          |  -          |
| TIP_RACK        | TipRack    |  Minimal    |  -          |
| PETRI_DISH      | ?          |  -          |  -          |
| Custom platform | ?          |  -          |  -          |
| Tools           | ?          |  -          |  -          |

Main points:

1. The first obvious difference is that Pipettin uses "tube rack" for everything, while
PLR only seems to have well "plates".
1. Second, I'm not sure PLR has an object counterpart to a petri dish.
2. PLR defines objects for wells and tip spots, while PW does not. These are inferred from the platform type, unless it's a custom platform with slots, or a petri dish (which does not really have predefined slots nor locations).
3. To be continued...


## Tip rack

¿How should I port PW (pronounced "pew" because its cool) objects to PLR objects?

Here is a relveant code chunk from `carrier_tests.py`:

In [ ]:
# From "carrier_tests.py"
from pylabrobot.resources.itemized_resource import create_equally_spaced
from pylabrobot.resources.ml_star.tip_creators import standard_volume_tip_with_filter

from pylabrobot.resources.carrier import Carrier, TipCarrier, create_homogenous_carrier_sites
from pylabrobot.resources.coordinate import Coordinate
from pylabrobot.resources.deck import Deck
from pylabrobot.resources.resource import Resource
from pylabrobot.resources.tip_rack import TipRack, TipSpot

tip_rack = TipRack( # pylint: disable=invalid-name
      name="custom tip rack",
      size_x=5, size_y=5, size_z=5,
      items=create_equally_spaced(TipSpot,
        dx=1, dy=1, dz=1,
        num_items_x=1, num_items_y=1, item_size_x=5, item_size_y=5,
        make_tip=standard_volume_tip_with_filter))

The `create_equally_spaced` seems specially important.

Paying attention to the `dy` and  `dx` arguments of the `create_equally_spaced` function. Those
arguments seem to define the full size of the platform.



In [ ]:
def create_equally_spaced(
    klass: Type[T],
    num_items_x: int, num_items_y: int,
    dx: float, dy: float, dz: float,
    item_size_x: float, item_size_y: float,
    **kwargs
) -> List[List[T]]:
  """ Make equally spaced resources.

  See :class:`ItemizedResource` for more details.

  Args:
    klass: The class of the resource to create
    num_items_x: The number of items in the x direction
    num_items_y: The number of items in the y direction
    dx: The bottom left corner for items in the left column
    dy: The bottom left corner for items in the top row
    dz: The z coordinate for all items
    item_size_x: The size of the items in the x direction
    item_size_y: The size of the items in the y direction
    **kwargs: Additional keyword arguments to pass to the resource constructor

  Returns:
    A list of lists of resources. The outer list contains the columns, and the inner list contains
    the items in each column.
  """
  pass

The `item_size_x` and `item_size_y` arguments to that function define how the items are spaced.

In most platforms this is set to the SBS spacing (9 mm, or its multiples).

The `create_equally_spaced` function defines an item's location with the following (familiar) math:

In [ ]:
item.location = Coordinate(x=dx + i * item_size_x, y=dy + (num_items_y-j-1) * item_size_y, z=dz)

> Note: it's interesting to note that the "tip" object in PLR is _not_ a PLR "resource".
>  
> Tubes, in contrast, must be resources (examples further down), just as Wells are.

Lets load the platforms and a workspace, and find the info for a tip rack:

In [ ]:
import json

# 'data/ws_export.json'
ws_export_file = 'data/pipettin-data-20240203/Workspaces.json'
with open(ws_export_file, 'r', encoding='utf-8') as f:
    workspaces = json.load(f)
workspace = workspaces[0]

pt_export_file = 'data/pipettin-data-20240203/Platforms.json'
with open(pt_export_file, 'r', encoding='utf-8') as f:
    platforms = json.load(f)

# Get a tip rack platform (the first one that shows up).
pew_tip_racks = [p for p in platforms if p["type"] == "TIP_RACK"]
pew_tip_rack = pew_tip_racks[0]

# Get a workspace item matching that platform.
pew_items = [i for i in workspace["items"] if i["platform"] == pew_tip_rack["name"] ]
pew_item = pew_items[0]

# Get the item's position in the workspace.
pew_item_pos = pew_item["position"]
pew_item_pos


Now convert the position to a PLR XYZ coordinate object:

In [ ]:
from pylabrobot.resources.coordinate import Coordinate

pew_item_location = Coordinate(**pew_item_pos)
pew_item_location

Now get the tip's data:

> Note that by using `create_equally_spaced` all tips must be identical.

In [ ]:
ct_export_file = 'data/pipettin-data-20240203/Containers.json'
with open(ct_export_file, 'r', encoding='utf-8') as f:
    containers = json.load(f)

pew_item_contents = pew_item["content"]
tip_content = pew_item_contents[0]

tip_container = [c for c in containers if c["name"]==tip_content["container"]][0]
tip_container

In [ ]:
# Get container offset
tip_container_offset = [o for o in pew_tip_rack["containers"] if o["container"] == tip_container["name"]][0]
tip_container_offset

Now we'll convert it to it's PLR conuterpart using `TipRack` and `create_equally_spaced`:

In [ ]:
from pylabrobot.resources.tip import Tip
from pylabrobot.resources.itemized_resource import create_equally_spaced

def make_pew_tip():
  """ Make single tip.

  Attributes from the Tip class:
    has_filter: whether the tip type has a filter
    total_tip_length: total length of the tip, in in mm
    maximal_volume: maximal volume of the tip, in ul
    fitting_depth: the overlap between the tip and the pipette, in mm
  """
  tip = Tip(
    has_filter=False,
    total_tip_length=tip_container["length"],
    maximal_volume=tip_container["maxVolume"],
    fitting_depth=tip_container["length"]-tip_container["activeHeight"]
  )

  return tip

tip_rack_item = TipRack(
    name=pew_item["name"],
    size_x=pew_tip_rack["width"],
    size_y=pew_tip_rack["length"],
    size_z=pew_tip_rack["height"],
    # category = "tip_rack", # The default.
    model=pew_tip_rack["name"], # Optional.
    items=create_equally_spaced(TipSpot,
      num_items_x=pew_tip_rack["wellsColumns"],
      num_items_y=pew_tip_rack["wellsRows"],
      # dx: The bottom left corner for items in the left column.
      dx=pew_tip_rack["firstWellCenterX"]-pew_tip_rack["wellSeparationX"]/2,
      # dy: The bottom left corner for items in the top row.
      dy=pew_tip_rack["firstWellCenterY"]-pew_tip_rack["wellSeparationY"]/2,
      # dz: The z coordinate for all items.
      # TODO: I dont know how "dz" is used later on. Check that it corresponds to activeHeight.
      dz=pew_tip_rack["activeHeight"],
      # XY distance between adjacent items in the grid.
      item_size_x=pew_tip_rack["wellSeparationX"],
      item_size_y=pew_tip_rack["wellSeparationY"],
      # TODO: This function should be replaced.
      # make_tip=standard_volume_tip_with_filter,
      make_tip=make_pew_tip
    ),
    with_tips=False
  )

print(tip_rack_item)

> Note: There is a "location" property set to `None`. It does not seem possible to set on instantiation.

### Reference implementations

- [ ] Inspect `pylabrobot/resources/ml_star/tip_creators.py`.

## Tube rack

There seem to be no abstractions for a "tube" in PLR. Furthermore, there seem to be no tube racks either. This is in line with the growing notion that robots can only use plates.

Pipettin does not care. It will stab a tube if its lid is closed (?).

Looking at `pylabrobot/resources/container.py`, there seems to be a useful `Container` class:

> "_A container is an abstract base class for a resource that can hold liquid._"

There is, however, an abstraction for a "well" at `pylabrobot/resources/well.py`, which subclasses Container. The only difference is that it assumes a cylindrical shape, and will estimate its maximum volume from the XZ dimensions on initialization.

It makes sense that I should write a Tube class.

There are some uncertainties though:

- Tubes can be moved, wells cannot.

### Issues with PLR

PLR does not have a "tube spot" object as it does for tips. If I wrote a "tube spot" object, I would't rally know how to do the tracking stuff.

I don't know how hard it would be to adapt the `itemized_resource` class to handle this scenario.

Furthermore, I wouldn't know how to insert only the needed tubes, thereby leaving empty spaces, and still be able to use the "A1:B2" syntax for selection.

This seems like a task for Rick/PLR folk.

Since no one else will be using tube racks, one possibility is to just write my own class entierly.

### My own class entirely

This makes sense in the context of selection by the usual pipettin stuff:

- name
- index
- label


In [ ]:
# TODO: re-write the TubeRack class without using itemized_resource, and without "A1:B2" selection.

In [ ]:
from pylabrobot.resources.pipettin.tube_racks import TubeRack, Tube

tube = Tube()

tube_rack = TubeRack(
  items=[[tube]]
)


## Trash bucket

> Due

## Petri Dish

> Due

## Custom platform

> Due

## Workspace

In [ ]:
pew_deck = SilverDeck(
  name=workspace["name"],
  # TODO: Update default size.
  size_x = workspace["width"],
  size_y = workspace["length"],
  size_z = workspace["height"],
  resource_assigned_callback = None,
  resource_unassigned_callback = None,
  # TODO: Update default origin.
  # origin = Coordinate(workspace["padding"]["left"], workspace["padding"]["right"], 0),
  origin = Coordinate(0, 0, 0),
  # TODO: Update default trash location.
  trash_location = Coordinate(x=82.84, y=53.56, z=5),
  no_trash = False
)

pew_deck

Lets assign the TipRack we created earlier to this custom Deck:

In [ ]:
pew_deck.assign_child_resource(resource=tip_rack_item, location=pew_item_location)

Inspect the contents:

In [ ]:
pew_deck.children

Yay!

# Testing out the SilverDeck 

> Have fun playing with my aberration.

## Setup

In [ ]:
# Example using exported data.
import json
import urllib.request

from pylabrobot.resources.pipettin.utils import load_objects

# path = "/tmp/workspaces.json"
location = 'https://gitlab.com/pipettin-bot/pipettin-gui/-/raw/develop/api/src/db/defaults/workspaces.json'
workspace = load_objects(location)[0]

# location = "/tmp/platforms.json"
location = 'https://gitlab.com/pipettin-bot/pipettin-gui/-/raw/develop/api/src/db/defaults/platforms.json'
platforms = load_objects(location)

# location = "/tmp/platforms.json"
location = 'https://gitlab.com/pipettin-bot/pipettin-gui/-/raw/develop/api/src/db/defaults/containers.json'
containers = load_objects(location)

In [ ]:
from pylabrobot.resources import SilverDeck

deck = SilverDeck(workspace, platforms, containers)

In [ ]:
deck.children
# deck.get_all_children()

In [ ]:
tiprack = deck.get_resource("200ul_tip_rack_MULTITOOL 1")

# Check if all tip spots have tips.
all([spot.tracker.has_tip for spot in tiprack.children])

In [ ]:
tiprack.print_grid()

## Check TipRack coordinates

In [ ]:
# A "location" is relative to the "parents".
deck.location, tiprack.location

In [ ]:
# An "absolute" location is relative to the deck's origin (the top-level object).
deck.get_absolute_location(), tiprack.get_absolute_location()

What about tip spots?

In [ ]:
tip_spot = tiprack.get_item("H1")

tip_spot.location, tip_spot.get_absolute_location()

Tips are not resources, but the have some dimensional properties:

In [ ]:
tip = tip_spot.get_tip()

tip.total_tip_length, tip.fitting_depth

### TipSpot Z calculation

Let's check item 1:

In [ ]:
tiprack_item = next(i for i in workspace["items"] if i["name"] == tiprack.name)

tipA1_content = next(t for t in tiprack_item["content"] if t["index"] == 1)

tip1A_container = next(c for c in containers if c["name"] == tipA1_content["container"])

tiprack_platform = next(p for p in platforms if p["name"] == tiprack_item["platform"])

container_offset_z = next(l["containerOffsetZ"]
                          for l in tiprack_platform["containers"]
                          if l["container"] == tip1A_container["name"])

tiprack_platform["activeHeight"], container_offset_z

Ahora calcular el Z del tip spot segun lo que dije:

In [ ]:
tip_spot_Z = tiprack_platform["activeHeight"] - container_offset_z

tip_spot_Z

Y comparar con lo que calculé en PLR:

In [ ]:
tip_spot.location.z == tip_spot_Z

Si fue "True" está todo bien ;)

### Well and TubeRack locations

Lets try with point 2.

Figure out coordinates for tubes and well plates.

### PLR-PW new child translator - Part 1: Deck

Lets try with point 3.

Have SilverDeck translate new child resources into PW objects. And maybe save them to the DB.

### PLR-PW new child translator - Part 2: Back

Lets try with point 4.

Have the backend translate (during setup) non-PW decks to PW format, and save them to the database.
   - This is essential when the deck is not PW-ish.

## Use with the Piper backend

In [ ]:
# Dummy backend.
# from pylabrobot.liquid_handling.backends.chatterbox_backend import ChatterBoxBackend
# back = ChatterBoxBackend()

# Piper backend.
from pylabrobot.liquid_handling.backends.piper_backend import PiperBackend
tools_url = 'https://gitlab.com/pipettin-bot/pipettin-gui/-/raw/develop/api/src/db/defaults/tools.json'
back = PiperBackend(config={"dry": True}, tool_defs=tools_url)

In [ ]:
from pylabrobot.liquid_handling import LiquidHandler

# TODO: Ask for a better error message when a non-instantiated backend is passed.
lh = LiquidHandler(backend=back, deck=deck)
await lh.setup()

In [ ]:
from pylabrobot.resources import set_tip_tracking, set_volume_tracking

# We enable tip and volume tracking globally using the `set_volume_tracking` and `set_tip_tracking` methods.
set_volume_tracking(enabled=True)
set_tip_tracking(enabled=True)

In [ ]:
tiprack = lh.get_resource("200ul_tip_rack_MULTITOOL 1")

tip_spots = tiprack["A1:B1"]

# TODO: Ask for a better error message when tips are passed instead of tip spots.
pickups = await lh.pick_up_tips(tip_spots, use_channels=[0, 1])

In [ ]:
tiprack.print_grid()

In [ ]:
try:
  # tips = tiprack.get_tip("A1")  # NOTE: For the PiperBackend.
  tips = tiprack.get_tips("A1:C1")
  print(tips)
except Exception as e:
  print(e)

In [ ]:
# plate = lh.get_resource("Standard 96-well plate 1")

tube_rack = lh.get_resource("5x16_1.5_rack 1")
tube_rack.print_grid()
tube_rack["A1"]

# tube_spots = plate["A1:H12"]
# tube_spots = tube_rack.children
#[spot.tracker.has_tube for spot in tube_spots]
#[spot.tracker.has_tube for spot in plate.get_all_items()]
#all([spot.tracker.has_tube for spot in tube_spots])
#len([spot.tracker.has_tube for spot in plate.get_all_items()])

tubes = tube_rack.get_all_tubes()
tubes

In [ ]:
[tube.tracker.liquids for tube in tubes]

In [ ]:
tube = tubes[0]

tube.tracker.is_disabled

In [ ]:
try:
  await lh.aspirate(tubes, vols=[100.0, 50.0, 200.0])
except Exception as e:
    print("Error:", e)

# try:
#   NOTE: Example for the PiperBackend.
#   for tube, vol in zip(tubes, [100.0, 50.0, 200.0]):
#     await lh.aspirate(tube, vol, tool_id="P20")
# except Exception as e:
#     print("Error:", e)

In [ ]:
try:
  await lh.aspirate(tubes, vols=[100.0, 50.0, 200.0])
except Exception as e:
    print(e)

[tube.tracker.liquids for tube in tubes]

In [ ]:
await lh.aspirate(tubes, vols=[90.0, 20.0, 9.0])

[tube.tracker.liquids for tube in tubes]

In [ ]:
await lh.dispense(tubes, vols=[1.0, 1.0, 1.0])

[tube.tracker.liquids for tube in tubes]

In [ ]:
# plate = lh.get_resource("plate_01")
# await lh.aspirate(plate["A1:C1"], vols=[100.0, 50.0, 200.0])
# await lh.dispense(plate["D1:F1"], vols=[100.0, 50.0, 200.0])

## Check Pickup coordinates

The "pickup" object is what a backend gets as argument to figure out the tip's coordinates.

Turns out it only contains the tip spot and an offset (and the tip, which can be accessed through the tip spot too).

In [ ]:
from pylabrobot.liquid_handling.standard import Pickup
from pylabrobot.resources.coordinate import Coordinate

tip_spot = tiprack.get_item("F1")

pickup = Pickup(resource=tip_spot, offset=Coordinate(0,0,0,), tip=tip_spot.get_tip())

vars(pickup)

## To-do List

In summary, I should:

- [x] Have the SilverDeck set the PLR TipSpot Z coordinate from PW data..
- [ ] Figure out coordinates for tubes and well plates.
- [ ] Have SilverDeck translate new child resources into PW objects. And maybe save them to the DB.
- [ ] Have the backend translate (during setup) non-PW decks to PW format, and save them to the database. This is essential when the deck is not PW-ish.


# Code form PLR notebooks

## STAR deck examples


Create a new liquid handler using `STAR` as its backend, and the hamilton "deck".

In [ ]:
# Basic imports:
# - liquid handler
from pylabrobot.liquid_handling import LiquidHandler
# - backend
from pylabrobot.liquid_handling.backends import STAR
from pylabrobot.liquid_handling.backends.chatterbox_backend import ChatterBoxBackend
# - deck
from pylabrobot.resources.hamilton import STARLetDeck

In [ ]:
from pylabrobot.resources import set_tip_tracking, set_volume_tracking

# We enable tip and volume tracking globally using the `set_volume_tracking` and `set_tip_tracking` methods.
set_volume_tracking(enabled=True)
set_tip_tracking(enabled=True)

In [ ]:
backend = ChatterBoxBackend()
# lh = LiquidHandler(backend=backend, deck=STARLetDeck())
lh = LiquidHandler(backend=backend, deck=STARLetDeck())

The final step is to open communication with the robot. This is done using the {func}`~pylabrobot.liquid_handling.LiquidHandler.setup` method.

In [ ]:
await lh.setup()


Now that we have a `LiquidHandler` instance, we can define the deck layout.

The layout in this tutorial will contain five sets of standard volume tips with filter, 1 set of 96 1mL wells, and tip and plate carriers on which these resources are positioned.

Start by importing the relevant objects and variables from the PyHamilton package. This notebook uses the following resources:

- {class}`~pylabrobot.resources.ml_star.tip_carriers.TIP_CAR_480_A00` tip carrier
- {class}`~pylabrobot.resources.ml_star.plate_carriers.PLT_CAR_L5AC_A00` plate carrier
- {class}`~pylabrobot.resources.corning_costar.plates.Cos_96_DW_1mL` wells
- {class}`~pylabrobot.resources.ml_star.tip_racks.HTF_L` tips

In [ ]:
from pylabrobot.resources import (
    TIP_CAR_480_A00,
    PLT_CAR_L5AC_A00,
    Cos_96_DW_1mL,
    HTF_L
)

Then create a tip carrier named `tip carrier`, which will contain tip rack at all 5 positions. These positions can be accessed using `tip_car[x]`, and are 0 indexed.

In [ ]:
tip_car = TIP_CAR_480_A00(name='tip carrier')
tip_car[0] = HTF_L(name='tips_01')

Use {func}`~pylabrobot.resources.abstract.assign_child_resources` to assign the tip carrier to the deck of the liquid handler. All resources contained by this carrier will be assigned automatically.

In the `rails` parameter, we can pass the location of the tip carrier. The locations of the tips will automatically be calculated.

In [ ]:
lh.deck.assign_child_resource(tip_car, rails=3)

Repeat this for the plates.

In [ ]:
plt_car = PLT_CAR_L5AC_A00(name='plate carrier')
plt_car[0] = Cos_96_DW_1mL(name='plate_01')

In [ ]:
lh.deck.assign_child_resource(plt_car, rails=15)

In [ ]:
from pylabrobot.resources.liquid import Liquid

plate = lh.get_resource("plate_01")
plate.set_well_liquids((Liquid.WATER, 300))

Let's look at a summary of the deck layout using {func}`~pylabrobot.liquid_handling.LiquidHandler.summary`.

In [ ]:
lh.summary()

## Picking up tips

Picking up tips is as easy as querying the tips from the tiprack.

In [ ]:
tiprack = lh.get_resource("tips_01")
await lh.pick_up_tips(tiprack["A1:C1"])

## Aspirating and dispensing

Aspirating and dispensing work similarly to picking up tips: where you use booleans to specify which tips to pick up, with aspiration and dispensing you use floats to specify the volume to aspirate or dispense in $\mu L$.

The cells below move liquid from wells `'A1:C1'` to `'D1:F1'` using channels 1, 2, and 3 using the {func}`~pylabrobot.liquid_handling.LiquidHandler.aspirate` and {func}`~pylabrobot.liquid_handling.LiquidHandler.dispense` methods.

In [ ]:
plate = lh.get_resource("plate_01")
await lh.aspirate(plate["A1:C1"], vols=[100.0, 50.0, 200.0])

After the liquid has been aspirated, dispense it in the wells below. Note that while we specify different wells, we are still using the same channels. This is needed because only these channels contain liquid, of course.

In [ ]:
await lh.dispense(plate["D1:F1"], vols=[100.0, 50.0, 200.0])

Let's move the liquid back to the original wells.

In [ ]:
await lh.aspirate(plate["D1:F1"], vols=[100.0, 50.0, 200.0])
await lh.dispense(plate["A1:C1"], vols=[100.0, 50.0, 200.0])

## Discarding tips

Finally, you can discard tips by using the {func}`~pylabrobot.liquid_handling.LiquidHandler.discard_tips` method.

In [ ]:
await lh.drop_tips(tiprack["A1:C1"])

In [ ]:
await lh.stop()